This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/intro.html).  I recommend having both windows open, side-by-side!

## Instructions for running this notebook on Deepnote

- Log in (the free account will be sufficient for this class)
- "Duplicate" the document.  Icon is in the top right next to Login.
- Read/run the cells one at a time OR use the "Run notebook" icon just above this cell
- To open the visualizer, click on the url printed just below "StartMeshcat" in the second code cell of the notebook.
- To teleop, use the keyboard commands printed in the third/fourth code cell outputs OR in meshcat, click "Open controls" and use the sliders.
- When you're done, press the ESCAPE key OR click the "Stop Simulation" button in meshcat.

In [1]:
import numpy as np
from pydrake.geometry import StartMeshcat
from pydrake.multibody.inverse_kinematics import (
    DifferentialInverseKinematicsParameters,
    DifferentialInverseKinematicsStatus,
    DoDifferentialInverseKinematics,
)
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder, EventStatus, LeafSystem
from pydrake.visualization import MeshcatPoseSliders

from manipulation import running_as_notebook
from manipulation.meshcat_utils import WsgButton
from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# Teleop Example (3D)

The physics and geometry engines running in the simulation above are actually running in 3D.  This example is almost identical, but we'll use the (default) 3D visualization and add more sliders for controlling the full `roll`, `pitch`, `yaw` angles and `x`, `y`, `z` positions of the end effector.

In [3]:
from constants import *
def make_stack():
    output = ''
    for i in range(NUM_LAYERS):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(NUM_BLOCKS_PER_LAYER):
            x = (BLOCK_WIDTH * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (BLOCK_WIDTH * (j - 1))
            z = BLOCK_HEIGHT * (i + 0.5) + TABLE_HEIGHT
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            print(f"block_{i * 3 + j}")
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

def construct_scenario():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_center
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: table_top_link
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [0.35, 0.5, 0.015]
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
"""
# plant_config:
#    time_step: 0.01

    driver_data = """
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""

    scenario_data += make_stack()
    scenario_data += driver_data
    scenario = load_scenario(data = scenario_data)
    return scenario

In [4]:
def teleop_3d():
    meshcat.ResetRenderMode()
    builder = DiagramBuilder()
    scenario = construct_scenario()
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    plant = station.GetSubsystemByName("plant")
    #station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    # TODO(russt): Replace with station.AddDiffIk(...)
    controller_plant = station.GetSubsystemByName(
        "iiwa.controller"
    ).get_multibody_plant_for_control()
    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"),
    )
    builder.Connect(
        differential_ik.get_output_port(),
        station.GetInputPort("iiwa.position"),
    )
    builder.Connect(
        station.GetOutputPort("iiwa.state_estimated"),
        differential_ik.GetInputPort("robot_state"),
    )

    # Set up teleop widgets.
    meshcat.DeleteAddedControls()
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            lower_limit=[0, -0.5, -np.pi, -0.6, -0.8, 0.0],
            upper_limit=[2 * np.pi, np.pi, np.pi, 0.8, 0.3, 1.1],
        )
    )
    builder.Connect(
        teleop.get_output_port(), differential_ik.GetInputPort("X_WE_desired")
    )
    # Note: This is using "Cheat Ports". For it to work on hardware, we would
    # need to construct the initial pose from the HardwareStation outputs.
    plant = station.GetSubsystemByName("plant")
    ee_pose = builder.AddSystem(
        ExtractBodyPose(
            station.GetOutputPort("body_poses"),
            plant.GetBodyByName("iiwa_link_7").index(),
        )
    )
    builder.Connect(
        station.GetOutputPort("body_poses"), ee_pose.get_input_port()
    )
    builder.Connect(ee_pose.get_output_port(), teleop.get_input_port())
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(
        wsg_teleop.get_output_port(0), station.GetInputPort("wsg.position")
    )

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.get_mutable_context()
    simulator_context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(simulator_context)
    for i in range(NUM_LAYERS*NUM_BLOCKS_PER_LAYER):
        some_body = (plant.GetBodyByName("jenga_block_link", plant.GetModelInstanceByName(f"block_{i}")))
        some_body.Lock(plant_context)
    block30 = plant.GetBodyByName("jenga_block_link", plant.GetModelInstanceByName(f"block_{30}"))
    block30.Unlock(plant_context) 
    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            print(simulator.get_context().get_continuous_state_vector())    
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d()

block_0
block_1
block_2
block_3
block_4
block_5
block_6
block_7
block_8
block_9
block_10
block_11
block_12
block_13
block_14
block_15
block_16
block_17
block_18
block_19
block_20
block_21
block_22
block_23
block_24
block_25
block_26
block_27
block_28
block_29
block_30
block_31
block_32
block_33
block_34
block_35
block_36
block_37
block_38
block_39
block_40
block_41
block_42
block_43
block_44
block_45
block_46
block_47
block_48
block_49
block_50
block_51
block_52
block_53
Press Space to open/close the gripper
Press Escape to stop the simulation


[-9.511088273249366e-05, 1.769809010982307e-05, -5.352200428148884e-05, 1.1947922374612965e-05, 1.8826614560279303e-05, -2.7702945301641987e-05, -0.0001411386098704521]
[0.0003606716671677142, 0.001164488478577219, -0.0007589068803192077, 0.0011176936296154957, -0.0011334012915719496, 0.0058284802476800774, -0.00037059861917304414]
[0.00810083175987911, 0.012569086074425612, 6.787572668269512e-05, 0.009739871006646424, 0.018927576605018318, 0.08906899974891884, -0.012032975493590147]


[0.016346762406376072, -0.004263594587667947, 0.0015769953504296429, -0.010295111814855817, 0.0006291327325057657, 0.09058872052072235, 0.002610213519475772]


[0.011393260686938208, 0.003665523296240368, 0.008351870077227083, 0.006119278921062125, -0.001168309702785394, 0.09618548328029361, -0.0006304434839224225]
[0.011857134161997209, 0.0010793282629805098, 0.006934151836307148, 0.00035923849137065787, -0.0008236309559510832, 0.0913011815530575, 0.0004957619778309061]
[0.011621429273825224, 0.001057697767603994, 0.006796164094616186, 0.00035162072287313366, -0.0008065535984312972, 0.08948950908703857, 0.00048636445077246854]
[0.01139085344826895, 0.001036712472270526, 0.006661324204579824, 0.00034464423782117884, -0.0007905510953127356, 0.08771398590913757, 0.00047671430050307326]
[0.011164852379588277, 0.0010161435373467432, 0.006529159615714232, 0.0003378061702322291, -0.0007748660916223405, 0.08597369012871975, 0.0004672556144799346]


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e32ce10-e406-4026-abfb-00be78c478cb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>